In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/binance-full-history/WAVES-USDT.parquet
/kaggle/input/binance-full-history/RLC-BNB.parquet
/kaggle/input/binance-full-history/AE-BTC.parquet
/kaggle/input/binance-full-history/BTC-GBP.parquet
/kaggle/input/binance-full-history/RVN-BNB.parquet
/kaggle/input/binance-full-history/DOTDOWN-USDT.parquet
/kaggle/input/binance-full-history/MATIC-BUSD.parquet
/kaggle/input/binance-full-history/HIVE-BTC.parquet
/kaggle/input/binance-full-history/DOT-BUSD.parquet
/kaggle/input/binance-full-history/XRP-USDC.parquet
/kaggle/input/binance-full-history/KAVA-USDT.parquet
/kaggle/input/binance-full-history/AXS-USDT.parquet
/kaggle/input/binance-full-history/STX-BTC.parquet
/kaggle/input/binance-full-history/XRP-PAX.parquet
/kaggle/input/binance-full-history/SHIB-BUSD.parquet
/kaggle/input/binance-full-history/CTXC-USDT.parquet
/kaggle/input/binance-full-history/TRX-USDT.parquet
/kaggle/input/binance-full-history/ANKR-BTC.parquet
/kaggle/input/binance-full-history/DOT-TRY.parquet
/kaggle/i

In [2]:
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
from sklearn.metrics import classification_report
import pyarrow.parquet as pq
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras import metrics
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv1D, TimeDistributed, Flatten, MaxPooling1D, MaxPooling2D, InputLayer, BatchNormalization, Conv2D
from tensorflow.keras import optimizers
import random

In [3]:
dataset = pq.read_pandas("/kaggle/input/binance-full-history/BTC-USDT.parquet").to_pandas()
dataset.sort_values("open_time")
dataset = dataset.resample("1D").apply(dict(open="first", high="max", low="min", close="last"))
dataset.columns = ["Open", "High", "Low", "Close"]
dataset = dataset.dropna()
dataset

,Open,High,Low,Close
open_time,,,,
2017-08-17,4261.479980,4485.390137,4200.740234,4285.080078
2017-08-18,4285.080078,4371.520020,3938.770020,4108.370117
2017-08-19,4108.370117,4184.689941,3850.000000,4139.979980
2017-08-20,4139.979980,4211.080078,4032.620117,4086.290039
2017-08-21,4069.129883,4119.620117,3911.790039,4016.000000
...,...,...,...,...
2022-02-03,36896.371094,37387.000000,36250.000000,37311.609375
2022-02-04,37311.980469,41772.328125,37026.730469,41574.250000
2022-02-05,41571.699219,41913.691406,40843.011719,41382.589844


In [4]:
def preprocess(df, window_size, valid):
  df = df.values
  buys = []
  holds = []
  sequences = []
  def prepare_batches(batch):
    label = batch[-1][-1]
    real_close = batch[-1][-2]
    batch_without_label = []
    for b in batch:
        b = np.delete(b, -1)
        batch_without_label.append(b)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_batch = scaler.fit_transform(batch_without_label)
    if label == 1:
        buys.append([scaled_batch, real_close, label])
    else:
        holds.append([scaled_batch, real_close, label])
    sequences.append([scaled_batch, real_close, label])
    return 1
    
  shape = (df.shape[0] - window_size + 1, window_size) + df.shape[1:]
  strides = (df.strides[0],) + df.strides
  batched_df = np.lib.stride_tricks.as_strided(df, shape=shape, strides=strides)
  mapped = list(map(prepare_batches, batched_df))
  
  if not valid:
      print("Buys: ", len(buys))
      print("Holds: ", len(holds))
      lower = min(len(buys), len(holds))
      buys = buys[-lower:]
      holds = holds[-lower:]
      print("New Buys: ", len(buys))
      print("New Holds: ", len(holds))
      print("-----------------------")
      sequences = buys + holds
      random.shuffle(sequences)

  X = []
  y = []
  c = []
  def seperate_sequences(sequence):
    X.append(sequence[0])
    c.append(sequence[1])
    y.append(sequence[2])
    return 1

  mapped = list(map(seperate_sequences, sequences))
  results = pd.DataFrame()
  results["close"] = c
  results["label"] = y
  return np.array(X), np.array(y), results

In [5]:
def labelize(df, sl, tp, candle_interval):
    df["sl"] = df.real_close*(1-(sl/100))
    df["tp"] = df.real_close*(1+(tp/100))

    df["tp_candle"] = 0
    df["sl_candle"] = candle_interval+2
    for i in range(1, candle_interval+1):
        df["prev_max_"+str(i)] = df.real_close.rolling(window=i).max()
        df["next_max_"+str(i)] = df["prev_max_"+str(i)].shift(-i)
        df["tp_candle"] = np.where((df["tp_candle"]==0)&(df["next_max_"+str(i)]>=df["tp"]), i, df["tp_candle"])

        df["prev_min_"+str(i)] = df.real_close.rolling(window=i).min()
        df["next_min_"+str(i)] = df["prev_min_"+str(i)].shift(-i)
        df["sl_candle"] = np.where((df["sl_candle"]==candle_interval+2)&(df["next_min_"+str(i)]<=df["sl"]), i, df["sl_candle"])

        df = df.drop(["next_max_"+str(i), "next_min_"+str(i), "prev_max_"+str(i), "prev_min_"+str(i)], axis=1)

    df["label"] = np.where((df["tp_candle"]!=0)&(df["tp_candle"]<df["sl_candle"]), 1, 0)
    print(f"{len(df.loc[df.label==1])} / {len(df)}")
    return df

In [6]:
def heikinashi(data, real_closes):
    df = data.copy()
    df['Close']=(df.Open + df.High + df.Low + df.Close)/4
    ha_open = [(df.Open[0] + df.Close[0])/2]
    [ha_open.append((ha_open[i]+df.Close.values[i])/2) for i in range(0, len(df)-1)]
    df['Open'] = ha_open
    df['High']=df[['Open','Close','High']].max(axis=1)
    df['Low']=df[['Open','Close','Low']].min(axis=1)
    df["real_close"] = real_closes
    return df

In [7]:
df = dataset.copy()
df = heikinashi(df, df.Close)
df = labelize(df, 0, 0.1, 100_000)
df = df.drop(["sl", "tp", "sl_candle", "tp_candle"], axis=1)
df

852 / 1636


,Open,High,Low,Close,real_close,label
open_time,,,,,,
2017-08-17,4284.826172,4485.390137,4200.740234,4308.172852,4285.080078,0
2017-08-18,4296.499512,4371.520020,3938.770020,4175.934570,4108.370117,1
2017-08-19,4236.217041,4236.217041,3850.000000,4070.760254,4139.979980,0
2017-08-20,4153.488647,4211.080078,4032.620117,4117.492676,4086.290039,0
2017-08-21,4135.490662,4135.490662,3911.790039,4029.135010,4016.000000,1
...,...,...,...,...,...,...
2022-02-03,37986.776023,37986.776023,36250.000000,36961.246094,37311.609375,1
2022-02-04,37474.011058,41772.328125,37026.730469,39421.324219,41574.250000,0
2022-02-05,38447.667639,41913.691406,38447.667639,41427.750000,41382.589844,1


In [8]:
train = df.iloc[:int(len(df)*0.9)]
test = df.iloc[int(len(df)*0.9):]
x_train, y_train, _ = preprocess(train, 3, False)
x_test, y_test, results = preprocess(test, 3, True)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(results.shape)

Buys:  769
Holds:  701
New Buys:  701
New Holds:  701
-----------------------
(1402, 3, 5)
(1402,)
(162, 3, 5)
(162,)
(162, 2)


In [9]:
model = Sequential()

model.add(LSTM(64, input_shape=(3, 5), return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["acc"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 3, 64)             17920     
_________________________________________________________________
dropout (Dropout)            (None, 3, 64)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 3, 64)             33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 64)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 6

In [10]:
history = model.fit(x_train, y_train, shuffle=True, validation_split=0.1, batch_size=32, epochs=25, verbose=1)

Epoch 1/25
40/40 [==============================] - 6s 42ms/step - loss: 0.6936 - acc: 0.4867 - val_loss: 0.6930 - val_acc: 0.5532
Epoch 2/25
40/40 [==============================] - 0s 8ms/step - loss: 0.6933 - acc: 0.4904 - val_loss: 0.6932 - val_acc: 0.4965
Epoch 3/25
40/40 [==============================] - 0s 8ms/step - loss: 0.6932 - acc: 0.5011 - val_loss: 0.6930 - val_acc: 0.4823
Epoch 4/25
40/40 [==============================] - 0s 8ms/step - loss: 0.6930 - acc: 0.5008 - val_loss: 0.6928 - val_acc: 0.5319
Epoch 5/25
40/40 [==============================] - 0s 8ms/step - loss: 0.6929 - acc: 0.5322 - val_loss: 0.6926 - val_acc: 0.5603
Epoch 6/25
40/40 [==============================] - 0s 8ms/step - loss: 0.6933 - acc: 0.5072 - val_loss: 0.6925 - val_acc: 0.5390
Epoch 7/25
40/40 [==============================] - 0s 8ms/step - loss: 0.6933 - acc: 0.4901 - val_loss: 0.6923 - val_acc: 0.5532
Epoch 8/25
40/40 [==============================] - 0s 8ms/step - loss: 0.6932 - acc: 0.5

In [11]:
predictions = model.predict(x_test)
binary_predictions = np.round(predictions).astype(int)
print(classification_report(y_test, binary_predictions, target_names=["HOLD", "LONG"]))

              precision    recall  f1-score   support

        HOLD       0.51      0.28      0.36        80
        LONG       0.51      0.74      0.61        82

    accuracy                           0.51       162
   macro avg       0.51      0.51      0.48       162
weighted avg       0.51      0.51      0.48       162

